# Wav2Vec

This was an introductory exercise to verify `wav2vec`'s functionality.

In [ ]:
import torch
torch.cuda.is_available()
torch.cuda.device_count()

## Pretrained model example

In [ ]:
import torch
import fairseq

cp_path = '/home/bayuan/Documents/ecog2txt_infonce/wav2vec_large.pt'#'/path/to/wav2vec.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()

wav_input_16khz = torch.randn(1,10000)
z = model.feature_extractor(wav_input_16khz)
c = model.feature_aggregator(z)

In [ ]:
print(c)

## Training a new model

### Prepare training data manifest

In [ ]:
import numpy as np
from scipy.io.wavfile import write

for i in range(30):
    wav_input_16khz = np.random.normal(0, 1, 10000)
    # print(wav_input_16khz.shape)
    write(f'/home/brian/Documents/ECE496/ecog2txt_infonce/testwavs/wav_{i}.wav', 16000, wav_input_16khz)

In [ ]:
!python3 /home/bayuan/Documents/fall23/fairseq/examples/wav2vec/wav2vec_manifest.py \
    /home/bayuan/Documents/fall23/ecog2vec/testwavs \
  --dest /home/bayuan/Documents/fall23/ecog2vec/manifest \
  --ext wav \
  --valid-percent 0.05

### Train a wav2vec model


In [ ]:
# !module load use.own
# !PYTHONPATH=/depot/jgmakin/data/conda_env/wav2vec/lib/python3.8/site-packages:/apps/cent7/xalt/site:/apps/cent7/xalt/libexec:/apps/spack/gilbreth/apps/intel-parallel-studio/cluster.2017.1-intel-17.0.1-2off4ih/advisor_2017.1.1.486553/pythonapi
# !echo $PYTHONPATH
# !module load conda-env/wav2vec-py3.8.5
!python3 -c 'import argparse; print(argparse.__file__)'
!python3 /home/bayuan/Documents/fall23/fairseq/train.py \
    /home/bayuan/Documents/fall23/ecog2vec/manifest \
  --save-dir /home/bayuan/Documents/fall23/ecog2vec/model \
  --num-workers 6 --fp16 --max-update 400000 --save-interval 1 --no-epoch-checkpoints \
  --arch wav2vec --task audio_pretraining --min-lr 1e-06 --stop-min-lr 1e-09 --optimizer adam --lr 0.001 --lr-scheduler cosine \
  --conv-feature-layers "[(512, 10, 5), (512, 8, 4), (512, 4, 2), (512, 4, 2), (512, 4, 2), (512, 1, 1), (512, 1, 1)]" \
  --conv-aggregator-layers "[(512, 2, 1), (512, 3, 1), (512, 4, 1), (512, 5, 1), (512, 6, 1), (512, 7, 1), (512, 8, 1), (512, 9, 1), (512, 10, 1), (512, 11, 1), (512, 12, 1), (512, 13, 1)]" \
  --skip-connections-agg --residual-scale 0.5 --log-compression --warmup-updates 500 --warmup-init-lr 1e-07 --criterion wav2vec --num-negatives 10 \
  --max-sample-size 150000 --skip-invalid-size-inputs-valid-test --max-epoch 20 --max-tokens 1500000 --batch-size 1

## Extract embeddings from the downstream task data


In [ ]:
!PYTHONPATH=/path/to/fairseq python examples/wav2vec/wav2vec_featurize.py --input /path/to/task/waves --output /path/to/output \
  --model /home/bayuan/Documents/ecog2txt_infonce/model/checkpoint_best.pt --split train valid test